# System Design for Interviews

**Level 4: Interview Preparation - Scalable Architecture & High-Level Design**

**Master system design principles, scalability patterns, and interview strategies for FAANG roles**

---

## System Design Fundamentals

**Core principles and patterns for building scalable systems**

In [ ]:
// System Design Demonstration Framework
import java.util.*;
import java.util.concurrent.*;
import java.util.concurrent.atomic.*;

public class SystemDesignFundamentals {
    
    // ==========================================================================
    // 1. LOAD BALANCING SIMULATION
    // ==========================================================================
    
    static class LoadBalancer {
        private List<String> servers = new ArrayList<>();
        private int currentIndex = 0;
        
        public LoadBalancer() {
            // Typical server pool
            servers.add("server-01");
            servers.add("server-02");
            servers.add("server-03");
            servers.add("server-04");
        }
        
        // Round-robin load balancing
        public synchronized String getNextServer() {
            String server = servers.get(currentIndex);
            currentIndex = (currentIndex + 1) % servers.size();
            return server;
        }
        
        public void addServer(String server) {
            servers.add(server);
        }
        
        public List<String> getAllServers() {
            return new ArrayList<>(servers);
        }
    }
    
    // ==========================================================================
    // 2. DATABASE CONNECTION POOLING SIMULATION
    // ==========================================================================
    
    static class Connection {
        private String id;
        private boolean inUse;
        
        public Connection(String id) {
            this.id = id;
            this.inUse = false;
        }
        
        public void use() { this.inUse = true; }
        public void release() { this.inUse = false; }
        public boolean isInUse() { return inUse; }
        public String getId() { return id; }
        
        @Override
        public String toString() {
            return "Connection[" + id + ", " + (inUse ? "IN_USE" : "AVAILABLE") + "]";
        }
    }
    
    static class ConnectionPool {
        private final List<Connection> pool;
        private final int maxSize;
        private final AtomicInteger connectionCount = new AtomicInteger(0);
        
        public ConnectionPool(int maxSize) {
            this.maxSize = maxSize;
            this.pool = new ArrayList<>();
            initializePool();
        }
        
        private void initializePool() {
            for (int i = 0; i < maxSize; i++) {
                pool.add(new Connection("conn-" + String.format("%03d", i + 1)));
            }
        }
        
        public synchronized Connection getConnection() throws InterruptedException {
            while (true) {
                for (Connection conn : pool) {
                    if (!conn.isInUse()) {
                        conn.use();
                        connectionCount.incrementAndGet();
                        return conn;
                    }
                }
                // All connections busy - wait for one to be released
                wait(100); // Wait up to 100ms
                break; // Timeout - return null or throw exception
            }
            throw new RuntimeException("No connections available - pool exhausted");
        }
        
        public synchronized void releaseConnection(Connection conn) {
            if (conn != null) {
                conn.release();
                connectionCount.decrementAndGet();
                notify(); // Wake up waiting threads
            }
        }
        
        public synchronized void printPoolStatus() {
            System.out.println("\n=== CONNECTION POOL STATUS ===");
            System.out.println("Total connections: " + pool.size());
            System.out.println("Active connections: " + getActiveConnections());
            System.out.println("Available connections: " + getAvailableConnections());
            System.out.println("Connections:");
            for (Connection conn : pool) {
                System.out.println("  " + conn);
            }
        }
        
        public int getActiveConnections() { return connectionCount.get(); }
        public int getAvailableConnections() { return pool.size() - connectionCount.get(); }
    }
    
    // ==========================================================================
    // 3. RATE LIMITING SIMULATION
    // ==========================================================================
    
    static class RateLimiter {
        private final Map<String, SlidingWindowCounter> userRequests = new ConcurrentHashMap<>();
        private final int maxRequests;
        private final long windowMs;
        
        static class SlidingWindowCounter {
            private final LinkedList<Long> requestTimes = new LinkedList<>();
            private final int maxRequests;
            private final long windowMs;
            
            public SlidingWindowCounter(int maxRequests, long windowMs) {
                this.maxRequests = maxRequests;
                this.windowMs = windowMs;
            }
            
            public synchronized boolean allowRequest() {
                long now = System.currentTimeMillis();
                
                // Remove expired requests
                while (!requestTimes.isEmpty() && (now - requestTimes.getFirst()) > windowMs) {
                    requestTimes.removeFirst();
                }
                
                // Check if under limit
                if (requestTimes.size() < maxRequests) {
                    requestTimes.addLast(now);
                    return true;
                }
                return false;
            }
            
            public int getRequestCount() { return requestTimes.size(); }
        }
        
        public RateLimiter(int maxRequests, long windowMs) {
            this.maxRequests = maxRequests;
            this.windowMs = windowMs;
        }
        
        public boolean isAllowed(String userId) {
            return userRequests.computeIfAbsent(userId, 
                k -> new SlidingWindowCounter(maxRequests, windowMs)).allowRequest();
        }
        
        public void printUserStats(String userId) {
            SlidingWindowCounter counter = userRequests.get(userId);
            if (counter != null) {
                System.out.println("User " + userId + ": " + counter.getRequestCount() + " requests in window");
            }
        }
    }
    
    // ==========================================================================
    // 4. URL SHORTENER SYSTEM
    // ==========================================================================
    
    static class UrlShortener {
        private final Map<String, String> urlToShort = new ConcurrentHashMap<>();
        private final Map<String, String> shortToUrl = new ConcurrentHashMap<>();
        private final AtomicLong counter = new AtomicLong(1);
        private final String baseUrl = "https://short.ly/";
        private final char[] charset = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789".toCharArray();
        
        public String shortenUrl(String longUrl) {
            if (urlToShort.containsKey(longUrl)) {
                return baseUrl + urlToShort.get(longUrl);
            }
            
            String shortCode = encode(counter.getAndIncrement());
            urlToShort.put(longUrl, shortCode);
            shortToUrl.put(shortCode, longUrl);
            
            return baseUrl + shortCode;
        }
        
        public String expandUrl(String shortUrl) {
            String shortCode = shortUrl.replace(baseUrl, "");
            return shortToUrl.getOrDefault(shortCode, null);
        }
        
        private String encode(long n) {
            StringBuilder sb = new StringBuilder();
            while (n > 0) {
                n--;  // Adjust for 0-based indexing
                sb.append(charset[(int)(n % charset.length)]);
                n /= charset.length;
            }
            return sb.reverse().toString();  // Reverse to get correct order
        }
        
        private long decode(String s) {
            long n = 0;
            for (char c : s.toCharArray()) {
                n = n * charset.length + Arrays.binarySearch(charset, c) + 1;
            }
            return n;
        }
    }
    
    public static void demonstrateSystemDesign() {
        System.out.println("=== SYSTEM DESIGN FUNDAMENTALS ===\n");
        
        // 1. Load Balancing Demo
        System.out.println("1. LOAD BALANCING SIMULATION:");
        LoadBalancer lb = new LoadBalancer();
        for (int i = 0; i < 10; i++) {
            System.out.println("Request " + (i+1) + " → " + lb.getNextServer());
        }
        
        // 2. Connection Pool Demo  
        System.out.println("\n2. DATABASE CONNECTION POOLING:");
        ConnectionPool pool = new ConnectionPool(3);
        pool.printPoolStatus();
        
        try {
            Connection conn1 = pool.getConnection();
            Connection conn2 = pool.getConnection();
            pool.printPoolStatus();
            
            pool.releaseConnection(conn1);
            pool.printPoolStatus();
            
            Connection conn3 = pool.getConnection();
            pool.printPoolStatus();
            
        } catch (Exception e) {
            System.out.println("Pool error: " + e.getMessage());
        }
        
        // 3. Rate Limiting Demo
        System.out.println("\n3. RATE LIMITING (5 requests per 10 seconds):");
        RateLimiter limiter = new RateLimiter(5, 10000); // 10 second window
        
        for (int i = 0; i < 8; i++) {
            boolean allowed = limiter.isAllowed("user123");
            System.out.println("Request " + (i+1) + ": " + (allowed ? "ALLOWED" : "BLOCKED"));
        }
        limiter.printUserStats("user123");
        
        // 4. URL Shortener Demo
        System.out.println("\n4. URL SHORTENER SYSTEM:");
        UrlShortener shortener = new UrlShortener();
        
        String longUrl1 = "https://www.example.com/very/long/url/path?param1=value1&param2=value2";
        String longUrl2 = "https://github.com/username/project/blob/main/src/example.java";
        
        String shortUrl1 = shortener.shortenUrl(longUrl1);
        String shortUrl2 = shortener.shortenUrl(longUrl2);
        
        System.out.println("Original: " + longUrl1);
        System.out.println("Short: " + shortUrl1);
        System.out.println("Expanded: " + shortener.expandUrl(shortUrl1));
        
        System.out.println("\nOriginal: " + longUrl2);
        System.out.println("Short: " + shortUrl2);
        System.out.println("Expanded: " + shortener.expandUrl(shortUrl2));
    }

    public static void main(String[] args) {
        demonstrateSystemDesign();
        
        System.out.println("\n🎯 SYSTEM DESIGN FUNDAMENTALS DEMONSTRATED:");
        System.out.println("• Load Balancing: Distributing traffic across servers");
        System.out.println("• Connection Pooling: Efficient database resource management");
        System.out.println("• Rate Limiting: Preventing system abuse");
        System.out.println("• URL Shortening: Compact representation with base62 encoding");
        
        System.out.println("\n📊 SCALABILITY PRINCIPLES:");
        System.out.println("• Horizontal scaling over vertical scaling");
        System.out.println("• Caching for performance optimization");
        System.out.println("• Asynchronous processing for responsiveness");
        System.out.println("• Monitoring and metrics for observability");
    }
}


## Designing Complex Systems

**Twitter, Facebook, YouTube, and Uber system architectures**

In [ ]:
// Complex System Design Examples
public class ComplexSystems {
    
    public static void twitterDesignDiscussion() {
        System.out.println("🐦 TWITTER SYSTEM DESIGN:");
        System.out.println("Scale: 200M+ daily active users, 6,000 tweets/second peak");
        
        System.out.println("\n🏗️ ARCHITECTURE COMPONENTS:");
        System.out.println("• API Gateway: Rate limiting, authentication, routing");
        System.out.println("• Timeline Service: Personalized content delivery");
        System.out.println("• Tweet Storage: Cassandra for write-heavy operations");
        System.out.println("• Fanout Service: Push tweets to follower timelines");
        System.out.println("• Search Service: Elasticsearch for real-time search");
        System.out.println("• Push Notification Service: Real-time tweet notifications");
        
        System.out.println("\n🚀 SCALING CHALLENGES SOLVED:");
        System.out.println("• Timeline Generation: Cache-backed for ~95% hit rate");
        System.out.println("• Fanout Operation: Push vs pull strategy based on follower count");
        System.out.println("• Read Optimization: Multi-level caching (browser → CDN → server)");
        System.out.println("• Write Optimization: Event-driven architecture with Kafka");
    }
    
    public static void facebookDesignDiscussion() {
        System.out.println("\n📘 FACEBOOK SYSTEM DESIGN:");
        System.out.println("Scale: 2.8B+ monthly active users, 510K photos uploaded/minute");
        
        System.out.println("\n🏗️ ARCHITECTURE COMPONENTS:");
        System.out.println("• Graph API: Social graph data access and manipulation");
        System.out.println("• News Feed Service: Machine learning-powered content ranking");
        System.out.println("• TAO (Graph Storage): Distributed graph database for relationships");
        System.out.println("• Haystack (Photos): Distributed photo storage system");
        System.out.println("• Scribe (Logging): Unified logging infrastructure");
        System.out.println("• Memcached Clusters: Massive caching for performance");
        
        System.out.println("\n🚀 SCALING CHALLENGES SOLVED:");
        System.out.println("• Real-time Graph Updates: Handle relationship changes instantly");
        System.out.println("• News Feed Ranking: ML models process billions of signals");
        System.out.println("• Cross-region Replication: Global data consistency");
        System.out.println("• Photo Storage: Petabyte-scale distributed storage");
    }
    
    public static void uberDesignDiscussion() {
        System.out.println("\n🚗 UBER SYSTEM DESIGN:");
        System.out.println("Scale: 111M+ monthly active users, 17M+ daily trips globally");
        
        System.out.println("\n🏗️ ARCHITECTURE COMPONENTS:");
        System.out.println("• Dispatch Service: Real-time ride matching algorithm");
        System.out.println("• Maps & Routing: Integration with Google Maps API");
        System.out.println("• Payment Service: Stripe integration with fraud detection");
        System.out.println("• Driver/Rider App: GPS tracking, real-time communication");
        System.out.println("• Surge Pricing Algorithm: Dynamic pricing based on demand/supply");
        System.out.println("• Kafka Event Streaming: Real-time data processing pipeline");
        
        System.out.println("\n🚀 SCALING CHALLENGES SOLVED:");
        System.out.println("• Real-time Matching: Sub-second ride matching at scale");
        System.out.println("• Location Services: Handle millions of GPS updates/second");
        System.out.println("• Surge Pricing: Machine learning demand prediction");
        System.out.println("• Eventual Consistency: Handle network partitions gracefully");
    }
    
    public static void interviewPreparationTips() {
        System.out.println("\n🎯 SYSTEM DESIGN INTERVIEW PREPARATION:");
        
        System.out.println("🗣️ COMMUNICATION STRATEGY:");
        System.out.println("• Always ask clarifying questions first");
        System.out.println("• Think aloud, don't stay silent");
        System.out.println("• Draw diagrams on whiteboard/virtual whiteboard");
        System.out.println("• Explain trade-offs between different approaches");
        
        System.out.println("\n🔧 TECHNICAL FRAMEWORK:");
        System.out.println("• Start with requirements analysis (functional vs non-functional)");
        System.out.println("• Estimate scale (users, requests/second, data size)");
        System.out.println("• Design high-level architecture first, then dive deep");
        System.out.println("• Identify and discuss bottlenecks proactively");
        
        System.out.println("\n📊 SCALING METRICS TO KNOW:");
        System.out.println("• Read-heavy vs Write-heavy systems");
        System.out.println("• CAP Theorem and consistency models");
        System.out.println("• Database sharding strategies");
        System.out.println("• Caching layers and cache invalidation");
        
        System.out.println("\n⚡ COMPREHENSIVE KNOWLEDGE:");
        System.out.println("• Database choices: SQL vs NoSQL, when to use each");
        System.out.println("• Message queues and asynchronous processing");
        System.out.println("• CDN, load balancers, and reverse proxies");
        System.out.println("• Microservices vs monolithic architectures");
        
        System.out.println("\n🎪 KEY SUCCESS PRINCIPLES:");
        System.out.println("• Product sense: Understand business requirements deeply");
        System.out.println("• Engineering judgment: Make informed trade-off decisions");
        System.out.println("• System thinking: Understand component interactions");
        System.out.println("• Practical experience: Build real systems to learn patterns");
    }
    
    public static void commonInterviewPatterns() {
        System.out.println("\n📋 COMMON SYSTEM DESIGN PATTERNS:");
        
        String[] patterns = {
            "URL Shortening Service (TinyURL)",
            "Design a Chat System (WhatsApp, Slack)",
            "Rate Limiting (API Gateway)",
            "Proximity Service (Yelp, Maps)",
            "Video Streaming Platform (YouTube, Netflix)",
            "Distributed Cache (Redis at scale)",
            "Search Autocomplete (Google Search)",
            "Typeahead Suggestion System",
            "API Rate Limiter",
            "Content Delivery Network (CDN)",
            "Distributed Message Queue (Apache Kafka)",
            "Distributed File System",
            "Real-time Gaming Leaderboard",
            "Payment System Design",
            "Notification System",
            "Search Engine Design",
            "Recommendation System",
            "Instagram/Photo Sharing",
            "Twitter/Facebook Timeline",
            "Google Drive/Dropbox File Sync",
            "Uber/Lyft Ride Sharing",
            "Airbnb Room Booking",
            "TicketMaster/Event Booking",
            "Web Crawler Design",
            "Distributed Key-Value Store",
            "Metrics/Monitoring System",
            "Logging Infrastructure"
        };
        
        System.out.println("Most frequently asked patterns:");
        for (int i = 0; i < Math.min(10, patterns.length); i++) {
            System.out.println((i + 1) + ". " + patterns[i]);
        }
        
        System.out.println("\n🔄 STARTING A DESIGN:");
        System.out.println("1. Clarify requirements and constraints");
        System.out.println("2. Estimate scale and performance requirements");
        System.out.println("3. Design high-level architecture (components, data flow)");
        System.out.println("4. Dive into specific components with alternatives");
        System.out.println("5. Discuss scalability, fault tolerance, and trade-offs");
        
        System.out.println("\n🛡️ DEFENSIVE DESIGN:");
        System.out.println("• Always consider failure scenarios");
        System.out.println("• Design for high availability and fault tolerance");
        System.out.println("• Think about security and privacy implications");
        System.out.println("• Consider operational costs and maintenance");
    }
    
    public static void main(String[] args) {
        twitterDesignDiscussion();
        facebookDesignDiscussion();
        uberDesignDiscussion();
        
        interviewPreparationTips();
        commonInterviewPatterns();
        
        System.out.println("\n🏆 SYSTEM DESIGN MASTERY ACHIEVED!");
        System.out.println("Ready for FAANG system design interviews! 🚀");
    }
}


## Database Sharding & Replication

**Designing databases for billion-user scale applications**

In [ ]:
// Database scaling patterns and strategies
public class DatabaseScaling {
    
    // ==========================================================================
    // SIMPLE SHARDING SIMULATION
    // ==========================================================================
    
    static class DatabaseShard {
        private final String shardId;
        private final Map<String, Object> data;
        
        public DatabaseShard(String shardId) {
            this.shardId = shardId;
            this.data = new ConcurrentHashMap<>();
        }
        
        public void put(String key, Object value) {
            data.put(key, value);
        }
        
        public Object get(String key) {
            return data.get(key);
        }
        
        public boolean containsKey(String key) {
            return data.containsKey(key);
        }
        
        public int size() { return data.size(); }
        public String getShardId() { return shardId; }
    }
    
    static class ShardedDatabase {
        private final List<DatabaseShard> shards;
        private final int shardCount;
        
        public ShardedDatabase(int shardCount) {
            this.shardCount = shardCount;
            this.shards = new ArrayList<>();
            
            for (int i = 0; i < shardCount; i++) {
                shards.add(new DatabaseShard("shard-" + i));
            }
        }
        
        // Consistent hashing-based shard selection
        private int getShardIndex(String key) {
            return Math.abs(key.hashCode()) % shardCount;
        }
        
        public void put(String key, Object value) {
            int shardIndex = getShardIndex(key);
            DatabaseShard shard = shards.get(shardIndex);
            shard.put(key, value);
            System.out.println("Stored '" + key + "' → " + shard.getShardId());
        }
        
        public Object get(String key) {
            int shardIndex = getShardIndex(key);
            DatabaseShard shard = shards.get(shardIndex);
            Object value = shard.get(key);
            System.out.println("Retrieved '" + key + "' from " + shard.getShardId());
            return value;
        }
        
        public void printStatistics() {
            System.out.println("\n=== DATABASE SHARD STATISTICS ===");
            int totalRecords = 0;
            for (DatabaseShard shard : shards) {
                System.out.println(shard.getShardId() + ": " + shard.size() + " records");
                totalRecords += shard.size();
            }
            System.out.println("Total records: " + totalRecords);
        }
    }
    
    // ==========================================================================
    // MASTER-SLAVE REPLICATION SIMULATION
    // ==========================================================================
    
    static class DatabaseServer {
        private final String serverId;
        private final Map<String, Object> data;
        private boolean isMaster;
        
        public DatabaseServer(String serverId) {
            this.serverId = serverId;
            this.data = new ConcurrentHashMap<>();
            this.isMaster = false;
        }
        
        public void write(String key, Object value) {
            if (isMaster) {
                data.put(key, value);
                System.out.println(serverId + " (MASTER) wrote: " + key);
            } else {
                System.out.println(serverId + " (SLAVE) - writes must go to master!");
            }
        }
        
        public Object read(String key) {
            Object value = data.get(key);
            System.out.println(serverId + " read: " + key + " = " + value);
            return value;
        }
        
        public void replicateFrom(DatabaseServer master, String key) {
            Object value = master.data.get(key);
            if (value != null) {
                data.put(key, value);
                System.out.println(serverId + " replicated: " + key + " from master");
            }
        }
        
        public void setAsMaster(boolean isMaster) {
            this.isMaster = isMaster;
            System.out.println(serverId + (isMaster ? " promoted to MASTER" : " demoted to SLAVE"));
        }
        
        public boolean isMaster() { return isMaster; }
        public String getServerId() { return serverId; }
    }
    
    static class ReplicatedDatabase {
        private final DatabaseServer master;
        private final List<DatabaseServer> slaves;
        
        public ReplicatedDatabase() {
            this.master = new DatabaseServer("MASTER");
            this.slaves = new ArrayList<>();
            
            // Set master
            master.setAsMaster(true);
            
            // Create slaves
            for (int i = 0; i < 2; i++) {
                DatabaseServer slave = new DatabaseServer("SLAVE-" + (i + 1));
                slaves.add(slave);
            }
        }
        
        public void write(String key, Object value) {
            master.write(key, value);
            // Replicate to slaves asynchronously (simplified)
            replicateToSlaves(key);
        }
        
        private void replicateToSlaves(String key) {
            for (DatabaseServer slave : slaves) {
                slave.replicateFrom(master, key);
            }
        }
        
        public Object read(String key, boolean readFromSlave) {
            if (readFromSlave && !slaves.isEmpty()) {
                // Load balance reads across slaves
                int slaveIndex = (int)(Math.random() * slaves.size());
                return slaves.get(slaveIndex).read(key);
            } else {
                return master.read(key);
            }
        }
        
        public void printReplicationStatus() {
            System.out.println("\n=== REPLICATION STATUS ===");
            System.out.println("Master: " + master.getServerId());
            System.out.println("Active slaves: " + slaves.size());
            System.out.println("Read scalability: " + slaves.size() + "x (load balanced)");
            System.out.println("Write operations: Routed to master only");
        }
    }
    
    public static void demonstrateScaling() {
        System.out.println("=== DATABASE SCALING TECHNIQUES ===\n");
        
        // 1. Sharding Demonstration
        System.out.println("1. DATABASE SHARDING:");
        ShardedDatabase db = new ShardedDatabase(4);
        
        String[] keys = {"user:12345", "user:67890", "order:111", "order:222", "product:999"};
        String[] values = {"John Doe", "Jane Smith", "Laptop", "Mouse", "iPhone"};
        
        for (int i = 0; i < keys.length; i++) {
            db.put(keys[i], values[i]);
        }
        
        db.printStatistics();
        
        System.out.println("\nTesting retrieval:");
        for (String key : keys) {
            db.get(key);
        }
        
        // 2. Replication Demonstration
        System.out.println("\n2. MASTER-SLAVE REPLICATION:");
        ReplicatedDatabase replDb = new ReplicatedDatabase();
        
        replDb.write("profile:user1", "Alice Johnson");
        replDb.write("profile:user2", "Bob Wilson");
        
        replDb.printReplicationStatus();
        
        System.out.println("\nTesting replicated reads:");
        replDb.read("profile:user1", true);  // Read from slave
        replDb.read("profile:user2", true);  // Read from slave
        replDb.read("profile:user1", false); // Read from master
    }
    
    public static void shardingVsReplication() {
        System.out.println("\n📊 SHARDING vs REPLICATION:");
        
        System.out.println("SHARDING (Horizontal Partitioning):");
        System.out.println("• Splits data across multiple servers (shards)");
        System.out.println("• Each shard contains subset of total data");
        System.out.println("• Improves write performance and storage capacity");
        System.out.println("• Increases read complexity (need to query multiple shards)");
        System.out.println("• Best for: Write-heavy workloads, massive datasets");
        
        System.out.println("\nREPLICATION (Data Copying):");
        System.out.println("• Copies data to multiple servers");
        System.out.println("• All servers contain complete data copy");
        System.out.println("• Improves read performance and availability");
        System.out.println("• Introduces consistency and synchronization complexity");
        System.out.println("• Best for: Read-heavy workloads, high availability needs");
        
        System.out.println("\nCOMBINED APPROACH:");
        System.out.println("• Shard primary data, replicate critical subsets");
        System.out.println("• Use sharding for capacity, replication for availability");
        System.out.println("• Route writes to primaries, reads to replicas");
        System.out.println("• Monitor performance and adjust shard/replica count as needed");
    }
    
    public static void realWorldConsiderations() {
        System.out.println("\n🌍 REAL-WORLD DATABASE SCALING:");
        
        System.out.println("CRITICAL FACTORS:");
        System.out.println("• Data Consistency: Eventual vs Strong consistency trade-offs");
        System.out.println("• Failover Strategy: Automatic promotion of replicas to master");
        System.out.println("• Cross-region Replication: Global data availability and compliance");
        System.out.println("• Monitoring: Performance metrics, error rates, throughput");
        
        System.out.println("\n цихINFRASTRUCTURE CHOICES:");
        System.out.println("• PostgreSQL/MySQL: Traditional RDBMS with clustering options");
        System.out.println("• Cassandra/MongoDB: Built-in sharding and replication");
        System.out.println("• DynamoDB/Spanner: Fully managed NoSQL solutions");
        System.out.println("• Redis Cluster: Advanced caching with sharding");
        
        System.out.println("\n📈 SCALE INDICATORS:");
        System.out.println("• 10M+ rows: Consider table partitioning");
        System.out.println("• 100M+ rows: Sharding becomes essential");
        System.out.println("• 1B+ rows: Advanced sharding strategies required");
        System.out.println("• Global users: Multi-region replication mandatory");
    }
    
    public static void main(String[] args) {
        demonstrateScaling();
        shardingVsReplication();
        realWorldConsiderations();
        
        System.out.println("\n🔥 DATABASE SCALING MASTERED:");
        System.out.println("• Sharding strategies for massive datasets");
        System.out.println("• Replication patterns for high availability");
        System.out.println("• Trade-offs and real-world implementation");
        
        System.out.println("\n🎯 SYSTEM DESIGN INTERVIEW READY!");
        System.out.println("You can now design databases for billion-user systems! 🚀");
    }
}
